In [1]:
with open('hkg_network_post.txt', mode='r') as BabyFlightNodes:
    FlightEdges = BabyFlightNodes.read().split("%SECTION%")[2]
with open('hkg_network_post.txt', mode='r') as BabyFlightNodes:
    FlightNodes = BabyFlightNodes.read().split("%SECTION%")[1]

In [2]:
with open('Flight1_HKG.txt', mode='r') as BabyFlight:
    Flight1 = BabyFlight.read().split("\n")[1]

In [3]:
from ipynb.fs.full.GetTrajectory import GetTrajectory
from ipynb.fs.full.PerpendicularDistance import NearestPointOnLine
from ipynb.fs.full.PerpendicularDistance import GetPerpendicularDistance
from ipynb.fs.full.PerpendicularDistance import EdgeSquareSearch, EdgeCircleSearch
from ipynb.fs.full.PerpendicularDistance import Haversine

0.026175739798742256


In [4]:
FlightTrajectoryNodes = GetTrajectory(Flight1)


In [5]:
from ipynb.fs.full.GetTaxiway import GetTaxiway
from ipynb.fs.full.GetTaxiway import GetEdgeID
from ipynb.fs.full.GetTaxiway import GetEdgeStart
from ipynb.fs.full.GetTaxiway import GetEdgeEnd
from ipynb.fs.full.GetTaxiway import GetEdgeLength
from ipynb.fs.full.GetTaxiway import GetEdgeSpecification
from ipynb.fs.full.GetTaxiway import GetEdgesAndNodes

In [9]:
FlightNodesList = GetTaxiway(FlightNodes, FlightEdges)[0]
FlightEdgesList = GetTaxiway(FlightNodes, FlightEdges)[1]
FlightEdgesWeights = GetTaxiway(FlightNodes, FlightEdges)[2]
FlightNodesList

[('1', '22.3174377', '113.9286655', '', 'intermediate'),
 ('2', '22.3161652', '113.9249003', '', 'intermediate'),
 ('3', '22.310662', '113.9341883', '', 'intermediate'),
 ('10', '22.295528', '113.8983679', '', 'intermediate'),
 ('11', '22.2959659', '113.9029503', '', 'intermediate'),
 ('12', '22.2960462', '113.9032229', '', 'intermediate'),
 ('13', '22.2963073', '113.903882', '', 'intermediate'),
 ('14', '22.2964844', '113.9063849', '', 'intermediate'),
 ('15', '22.2967974', '113.9053643', '', 'intermediate'),
 ('17', '22.2968212', '113.9055166', '', 'intermediate'),
 ('19', '22.2969162', '113.9062138', '', 'intermediate'),
 ('22', '22.2969909', '113.8978289', '', 'intermediate'),
 ('24', '22.2969941', '113.9060058', '', 'intermediate'),
 ('25', '22.2970048', '113.9060926', '', 'intermediate'),
 ('27', '22.2970505', '113.9062468', '', 'intermediate'),
 ('28', '22.2971123', '113.9063613', '', 'intermediate'),
 ('32', '22.297488', '113.9049467', '', 'intermediate'),
 ('33', '22.2975312',

In [7]:
EdgesAndNodes = GetEdgesAndNodes(FlightEdgesList)
#EdgeID[0], NodeStart[1], NodeFinish[2], EdgeLength[3], EdgeSpecification[4], StartX[5], StartY[6], EndX[7], EndY[8]
EdgesAndNodes

IndexError: list index out of range

In [ ]:
#The Algorithm
for item in FlightTrajectoryNodes:
    print(item)
    PointX = float(item[0])
    PointY = float(item[1])
    #Iterate through the Edges for each Trajectory Node
    for index, edge in enumerate(EdgesAndNodes):
        try:
        EdgeID = edge[0]
        EdgeStart = edge[1]
        EdgeEnd = edge[2]
        EdgeLength = edge[3]
        EdgeType = edge[4]
        
        StartX = edge[5]
        StartY = edge[6]
        EndX = edge[7]
        EndY = edge[8]
        except indexing error:
            break
        #This will find edge neighbors to get input variables from
        if EdgeSquareSearch(PointX, PointY, StartX, StartY, EndX, EndY, 10) == True:
            #Input Variables: Euclidian Distance: IntX and IntY would be the Node snapping in place
            (IntX, IntY) = NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY)
            PDistance = GetPerpendicularDistance(PointX, PointY, (IntX, IntY)[0], (IntX, IntY)[1])
            print(edge)
            print(index, EdgeID, StartX, StartY, EndX, EndY, (IntX,IntY), PDisthgkance)
        else:
            pass